In [1]:
import numpy as np
import functools as ft
import more_itertools
from more_itertools import distinct_permutations as idp
from scipy.linalg import expm

In [2]:
# down -- ground state
# up -- excited state

down = [1j, 1] / np.sqrt(2)
up = [-1j, 1] / np.sqrt(2)

one = np.array([[1 , 0], [0, 1]])
sz = np.array([[1 , 0], [0, -1]]) 
plus = np.array([[0 , 1], [0, 0]])
minus = np.array([[0 , 0], [1, 0]])
mplus = plus @ sz
mminus = sz @ minus

def psi(k, n):
    listvec = [0] * (n - k) + [1] * k
    f = []
    for i in idp(listvec):
        v = list(i)
        for j in range(len(v)):
            if v[j] == 0:
                v[j] = down            
            else:
                v[j] = up          
        vec = ft.reduce(np.kron, v)
        f.extend([vec])
    return f

In [3]:
def perm_SYK2(n):
    listvec = ['P', 'M']  + ['I'] * (n - 2)  
    L1 = []
    L2 = []
    k = 0
    for i in idp(listvec):
        k += 1
        v = list(i)
        vconj = [0] * len(listvec)        
        for j in range(len(listvec)):
            if v[j] == 'P':
                vconj[j] = 'M'        
            elif v[j] == 'M':
                vconj[j] = 'P'   
            else:
                vconj[j] = 'I'    
        L1.extend([v])
        L2.extend([vconj])
        if k > 1:
            for m in range(2, k+1):
                if v == L2[m-2]:
                    L1.remove(v)
    l = []    
    for k in range(len(L1)):
        for m in range(len(L1[k])):
            if L1[k][m] == 'P':
                L1[k][m] = 'PZ'
                l.extend([m])
                break
            elif L1[k][m] == 'M':
                L1[k][m] = 'ZM'
                l.extend([m])
                break  
    for k in range(len(L1)):
        for m in range(l[k]+1,len(L1[k])):
            if L1[k][m] == 'I':
                L1[k][m] = 'Z'
            elif L1[k][m] == 'P':
                break
            elif L1[k][m] == 'M':
                break
    return L1

def J_SYK2(n, J):
    varJ = J / n 
    Js = [] 
    L1 = perm_SYK2(n)
    for i in range(len(L1)):    
        v = 4. * np.random.normal(loc = 0.0, scale = 0.5 * varJ, size = (1, 2)).view(np.complex)[0][0]
        Js.extend([v])
    return Js  

def H_SYK2(n, couplings):
    dim = (2 ** n, 2 ** n)
    H_sum = np.zeros(dim)
    L1 = perm_SYK2(n)
    Js = couplings
    k = 0
    for i in range(len(L1)):
        k += 1
        v = L1[i]        
        vconj = [0] * len(v)  
        for j in range(len(v)):
            if v[j] == 'ZM':
                v[j] = mminus
                vconj[j] = mplus
            elif v[j] == 'P':
                v[j] = plus
                vconj[j] = minus
            if v[j] == 'PZ':
                v[j] = mplus 
                vconj[j] = mminus
            elif v[j] == 'M':
                v[j] = minus
                vconj[j] = plus
            elif v[j] == 'Z':
                v[j] = sz
                vconj[j] = sz
            else:
                v[j] = one
                vconj[j] = one       
        H =  Js[k-1] * ft.reduce(np.kron, v) + np.conjugate(Js[k-1]) * ft.reduce(np.kron, vconj)
        H_sum = H_sum + H
        #for i in range(0,dim[0]):
        #    for j in range(0,dim[1]):
        #        if H_sum[i][j] - np.conjugate(H_sum[j][i]) != 0.:
        #            print('Hermicity check fails!')                             
    return H_sum

def evolved_state(n, t, couplings, H):
    Js = couplings
    gs = psi(0, n)[0]
    U = expm(- 1j * t * H)
    f = np.dot(U, gs)
    return f

def p(n, t, states, couplings, H):
    pop_sum = 0.
    fs = states
    Js = couplings
    es = evolved_state(n, t, Js, H)
    for i in range(len(fs)):
        overlap = np.dot(np.conjugate(fs[i]), es)
        pop = np.abs(overlap) ** 2
        pop_sum = pop_sum + pop
    return pop_sum

In [4]:
num = 4
Jc = 1

time = np.linspace(0, 9, 10) 
nr = 20 # -- number of realizations

Lk = [ [0] * nr for t in range(len(time))] 
Lk2 = [ [0] * nr for t in range(len(time))] 
Lp = [ [0] * nr for t in range(len(time))] 
S = [ [0] * nr for t in range(len(time))] 
S2 = [ [0] * nr for t in range(len(time))] 
SS2 = [ [0] * nr for t in range(len(time))]

for j in range(nr):
    print(j)
    Js = J_SYK2(num, Jc)
    H2 = H_SYK2(num, Js)
    for t in range(len(time)):
        #print(time[t])
        ks = []
        ps = []
        for k in range(num+1):
            fs = psi(k, num)
            ks.append(k)
            ps.append(p(num, time[t], fs, Js, H2))    
        Lk[t][j] = ks
        Lk2[t][j] = [q ** 2 for i, q in enumerate(ks)]
        Lp[t][j] = ps
        S[t][j] = - num / 2 + np.array(Lk[t][j]) @ np.array(Lp[t][j]) 
        S2[t][j] = np.array(Lk2[t][j]) @ np.array(Lp[t][j]) 
        SS2[t][j] =  S2[t][j] - ( S[t][j] + num / 2  ) ** 2

S0 = np.array([- num / 2 for i in range(len(time))])  
pav = np.sum(np.array(Lp), 1) / nr
Sav = np.sum(np.array(S), 1)  / nr
S2av = np.sum(np.array(S2), 1)  / nr
VarS = S2av  - (Sav - S0) ** 2

np.save('data/time.npy', time, allow_pickle = True)
np.save('data/p={}_nr={}.npy'.format(num, nr), Lp, allow_pickle = True)
np.save('data/pav={}_nr={}.npy'.format(num, nr), pav, allow_pickle = True)
np.save('data/S_N={}_nr={}.npy'.format(num, nr), S, allow_pickle = True)
np.save('data/SS2_N={}_nr={}.npy'.format(num, nr), SS2, allow_pickle = True)
np.save('data/Sav_N={}_nr={}.npy'.format(num, nr), Sav, allow_pickle = True)
np.save('data/S2av_N={}_nr={}.npy'.format(num, nr), S2av, allow_pickle = True)
np.save('data/VarS_N={}_nr={}.npy'.format(num, nr), VarS, allow_pickle = True)

0
1
2
3
4


C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Nick\Anaconda3\lib\si

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
